In [3]:
class Queue:
    def __init__(self):
        self.items=[]
    def enqueue(self,item):
        self.items.insert(0,item)
    def dequeue(self):
        return self.items.pop()
    def isEmpty(self):
        return len(self.items)==0
    def size(self):
        return len(self.items)

In [4]:
import re

def cleantxt(a):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', a)
    return cleantext
    

In [5]:
# Print iterations progress
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█'):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print('\r%s |%s| %s%% %s' % (prefix, bar, percent, suffix), end = '\r')
    # Print New Line on Complete
    if iteration == total: 
        print()

In [23]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
def chem_search(s_name):    
    s_link='https://pubchem.ncbi.nlm.nih.gov/#query='+s_name
    options_1 = webdriver.ChromeOptions()
    options_1.add_argument('headless')
    options_1.add_argument('window-size=1920x1080')
    options_1.add_argument("disable-gpu")
    driver = webdriver.Chrome('chromedriver.exe',options=options_1)
    #driver = webdriver.Chrome('chromedriver.exe')
    driver.get(s_link)
    # Initial call to print 0% progress
    printProgressBar(0, 50, prefix = 'Progress:', suffix = 'Complete', length = 50)
    for i in range(50):
        # Do stuff...
        time.sleep(0.1)
        # Update Progress Bar
        printProgressBar(i + 1, 50, prefix = '검색 중:', suffix = 'Complete', length = 50)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    a = soup.select("#collection-results-container > div > div > div:nth-child(2) > ul > li:nth-child(1) > div > div > div.p-sm.flex-container.flex-nowrap.width-100 > div.flex-grow-1.p-md-left > div.f-medium.p-sm-top.p-sm-bottom.f-1125 > a")
    b=a[0].get('href')
    c=Queue()
    temp=0
    cidnum=''
    for i in b:
        if temp==4:
            c.enqueue(i)
        if i=='/':
            temp+=1
    while not c.isEmpty():
        cidnum += c.dequeue()

    #cid_link='https://pubchem.ncbi.nlm.nih.gov/rest/pug_view/data/compound/'+cidnum+'/XML/?response_type=display'
    cid_link='https://pubchem.ncbi.nlm.nih.gov/compound/'+cidnum+'#section=Safety-and-Hazards'
    driver.get(cid_link)
    for i in range(300):
        time.sleep(0.1)
        printProgressBar(i + 1, 300, prefix = '데이터 수집 중:', suffix = 'Complete', length = 50)
    html2 = driver.page_source
    soup2 = BeautifulSoup(html2, 'html.parser')
    
    #x1=soup2.select("#GHS-Classification > div.section-content > div.section-content-item > div.overflow-x-auto.border > table > tbody > tr:nth-child(3) > td > div")
    return soup2

In [12]:
from bs4 import BeautifulSoup
def finding_hazard(a):
    x=a.select("#GHS-Classification > div.section-content > div.section-content-item > div.overflow-x-auto.border > table > tbody > tr:nth-child(3) > td > div")
    cleanres=[]
    for i in x[0].find_all('p'):
        cleanres.append(i.text)
    return cleanres    
def finding_formula(a):
    x=a.select("#main-content > div.main-width > div > div > div.summary.p-l-top.p-l-bottom > div > table > tbody > tr:nth-child(5) > td")
    b=x[0].find('a')
    e=b.get('href')
    c=0
    d=''
    for i in e:
        c+=1
        if c>8:
            d+=i
    return d       
def finding_name(a):
    x=a.select("#main-content > div.main-width > div > div > div.summary.p-l-top.p-l-bottom > div > table > tbody > tr:nth-child(7) > td > div > div")
    cleanres=[]
    for i in x[0].find_all('p'):
        cleanres.append(i.text)
    return cleanres 
def finding_mweight(a):
    x=a.select("#main-content > div.main-width > div > div > div.summary.p-l-top.p-l-bottom > div > table > tbody > tr:nth-child(8) > td")
    return x[0].text

In [8]:
def mainf(a):
    res=chem_search(a)
    mres=[]
    mres.append('화학식:'+finding_formula(res))
    mres.append('분자량:'+finding_mweight(res))
    mres.append('이름')
    for i in finding_name(res):
        mres.append(i)
    mres.append('위험성')
    for i in finding_hazard(res):
        mres.append(i)
    return mres    

In [25]:
b=input('화합물의 이름을 입력하세요')
a=mainf(b)
for i in a:
    print(i)

화합물의 이름을 입력하세요C6H6
검색 중: |██████████████████████████████████████████████████| 100.0% Completete
데이터 수집 중: |██████████████████████████████████████████████████| 100.0% Complete
화학식:C6H6
분자량:78.114 g/mol
이름
benzene
benzol
71-43-2
benzole
Cyclohexatriene
위험성

H225: Highly Flammable liquid and vapor [Danger Flammable liquids]
H304: May be fatal if swallowed and enters airways [Danger Aspiration hazard]
H315: Causes skin irritation [Warning Skin corrosion/irritation]
H319: Causes serious eye irritation [Warning Serious eye damage/eye irritation]
H340: May cause genetic defects [Danger Germ cell mutagenicity]
H350: May cause cancer [Danger Carcinogenicity]
H372 **: Causes damage to organs through prolonged or repeated exposure [Danger Specific target organ toxicity, repeated exposure]



In [ ]:
from googletrans import Translator

